<a href="https://colab.research.google.com/github/darisoy/EE517_Sp21/blob/master/hw3/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🐍 Setup Python environment

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 28.6MB/s 
     |████████████████████████████████| 3.3MB 44.5MB/s 
     |████████████████████████████████| 901kB 49.7MB/s 


In [2]:
import numpy as np
import pandas as pd
import math
import torch
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from tokenizers import decoders
from sklearn.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 📀 Load the data

In [3]:
labels = {'O' : 0,
          'B-geo-loc' : 1,
          'I-geo-loc' : 2,
          'B-product' : 3,
          'I-product' : 4,
          'B-facility' : 5,
          'I-facility' : 6,
          'B-company' : 7,
          'I-company' : 8,
          'B-person' : 9,
          'I-person' : 10,
          'B-sportsteam' : 11,
          'I-sportsteam' : 12,
          'B-musicartist' : 13,
          'I-musicartist' : 14,
          'B-movie' : 15,
          'I-movie' : 16,
          'B-tvshow' : 17,
          'I-tvshow' : 18,
          'B-other' : 19,
          'I-other' : 20,
          }
end_token = '<END>'
beg_token = '<BEG>'

In [4]:
def get_sentences(df):
    sentences = []
    labels = []
    running_sentence = ''
    runnnig_label = []
    for idx, row in df.iterrows():
        if row.word == end_token:
            if len(running_sentence[:-1]) > 0:
                sentences.append(running_sentence[:-1])
                labels.append(runnnig_label)
            running_sentence = ''
            runnnig_label = []
        else:
            running_sentence += row.word + ' '
            runnnig_label.append(row.tag)
    return sentences, labels

def get_data(type):
    data = pd.read_csv('https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/data/' + type, delimiter='\t', names=["word", "tag"], skip_blank_lines=False, quoting=3)
    data = data.fillna({'word': end_token, 'tag': 'O'})
    data.tag = data.tag.apply((lambda x: labels[x]))
    return get_sentences(data)

# 🔐 Encode the data using BERT transformer

## Load the transformer

In [5]:
transformer_name = "distilbert-base-uncased"
transformer = DistilBertModel.from_pretrained(transformer_name)
tokenizer = DistilBertTokenizer.from_pretrained(transformer_name)
tokenizer.decoder = decoders.WordPiece()

## Get dataset embeddings

In [6]:
def get_sublist_start_end(sl, l):
    results = []
    sll = len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if tokenizer.decode(l[ind:ind+sll]) ==  tokenizer.decode(sl):
            results.append([ind,ind+sll-1])
    return results

def get_embeddings(sentences):
    transformer.eval()
    transformer.to(device)
    data = []
    with torch.no_grad():
        for sentence in tqdm(sentences):
            tokens = tokenizer.encode(sentence)
            out = transformer(torch.tensor(tokens).unsqueeze(0).to(device))
            embed = []
            for i, word in enumerate(sentence.split()):
                target = word
                target_ids = tokenizer.encode(target, add_special_tokens=False)
                start_end_found = get_sublist_start_end(target_ids , tokens)
                if len(start_end_found) > 1:
                    occurrences_pos_of_cur_word = [word_i for word_i, x in enumerate(sentence.split()) if x == word]
                    cur_occ_num =  occurrences_pos_of_cur_word.index(i)
                    target_idx = start_end_found[cur_occ_num]
                else:
                    target_idx = start_end_found[0]
                embed.append(torch.mean(out[0][0][target_idx[0]:target_idx[1]+1], 0))
            data.append(torch.stack(embed))
    return data

In [7]:
train_sentences, train_tags = get_data('train')
train_embeddings = get_embeddings(train_sentences)

In [8]:
valid_sentences, valid_tags = get_data('dev')
valid_embeddings = get_embeddings(valid_sentences)

In [9]:
test_sentences, test_tags = get_data('test')
test_embeddings = get_embeddings(test_sentences)

# 🧑‍💻 Classify the embeddings using RNN

In [10]:
# Model Definition
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.hidden_dim = 128
        self.num_layers = 2
        self.rnn = nn.GRU(768, self.hidden_dim, self.num_layers, batch_first=True) 
        self.fc = nn.Linear(self.hidden_dim, len(labels))
    
    def forward(self, x):
        h = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        out, (h) = self.rnn(x, (h.detach()))
        out = self.fc(out) 
        return out

In [11]:
classifier = RNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
epochs = 10

In [12]:
# train
for epoch in range(epochs):
    running_loss = 0.0
    for i, sentence in enumerate(tqdm(train_embeddings)):
        tags = torch.tensor(train_tags[i])
        sentence, tags = sentence.to(device), tags.to(device)
        optimizer.zero_grad()
        outputs = classifier(sentence.unsqueeze(dim=0))
        loss = criterion(outputs.squeeze(dim=0), tags)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('[Epoch %d]\tTrain Loss: \t\t%.3f' % (epoch+1, running_loss / len(train_embeddings)))


[Epoch 1]	Train Loss: 		0.224



[Epoch 2]	Train Loss: 		0.142



[Epoch 3]	Train Loss: 		0.105



[Epoch 4]	Train Loss: 		0.076



[Epoch 5]	Train Loss: 		0.056



[Epoch 6]	Train Loss: 		0.042



[Epoch 7]	Train Loss: 		0.033



[Epoch 8]	Train Loss: 		0.026



[Epoch 9]	Train Loss: 		0.022



[Epoch 10]	Train Loss: 		0.021


# 📈 Evaluate valid and test data

## Helper functions

In [13]:
def evaluate(sentences, sentence_tags):
    correct = 0
    total = 0
    running_loss = 0
    truth = []
    preds = []
    for i, sentence in enumerate(tqdm(sentences)):
        tags = torch.tensor(sentence_tags[i])
        sentence, tags = sentence.to(device), tags.to(device)
        optimizer.zero_grad()

        outputs = classifier(sentence.unsqueeze(dim=0)).squeeze(dim=0)
        pred = outputs.squeeze(dim=1).argmax(dim=1)
        loss = criterion(outputs, tags)
        running_loss += loss.item()

        correct += torch.sum(tags == pred)
        total += len(tags)

        for t in sentence_tags[i]:
            truth.append(t)
        for p in pred:
            preds.append(p.item())

    print('Overall Accuracy: \t%.3f%% \tloss: %.3f' % (100*correct/total, running_loss/len(sentences)))
    return preds, truth

In [14]:
import sys
from collections import defaultdict

def split_tag(chunk_tag):
    """
    split chunk tag into IOBES prefix and chunk_type
    e.g. 
    B-PER -> (B, PER)
    O -> (O, None)
    """
    if chunk_tag == 0:
        return ('O', None)
    return list(labels.keys())[chunk_tag].split('-', maxsplit=1)

def is_chunk_end(prev_tag, tag):
    """
    check if the previous chunk ended between the previous and current word
    e.g. 
    (B-PER, I-PER) -> False
    (B-LOC, O)  -> True
    Note: in case of contradicting tags, e.g. (B-PER, I-LOC)
    this is considered as (B-PER, B-LOC)
    """
    prefix1, chunk_type1 = split_tag(prev_tag)
    prefix2, chunk_type2 = split_tag(tag)

    if prefix1 == 'O':
        return False
    if prefix2 == 'O':
        return prefix1 != 'O'

    if chunk_type1 != chunk_type2:
        return True

    return prefix2 in ['B', 'S'] or prefix1 in ['E', 'S']

def is_chunk_start(prev_tag, tag):
    """
    check if a new chunk started between the previous and current word
    """
    prefix1, chunk_type1 = split_tag(prev_tag)
    prefix2, chunk_type2 = split_tag(tag)

    if prefix2 == 'O':
        return False
    if prefix1 == 'O':
        return prefix2 != 'O'

    if chunk_type1 != chunk_type2:
        return True

    return prefix2 in ['B', 'S'] or prefix1 in ['E', 'S']


def calc_metrics(tp, p, t, percent=True):
    """
    compute overall precision, recall and FB1 (default values are 0.0)
    if percent is True, return 100 * original decimal value
    """
    precision = tp / p if p else 0
    recall = tp / t if t else 0
    fb1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    if percent:
        return 100 * precision, 100 * recall, 100 * fb1
    else:
        return precision, recall, fb1


def count_chunks(true_seqs, pred_seqs):
    """
    true_seqs: a list of true tags
    pred_seqs: a list of predicted tags
    return: 
    correct_chunks: a dict (counter), 
                    key = chunk types, 
                    value = number of correctly identified chunks per type
    true_chunks:    a dict, number of true chunks per type
    pred_chunks:    a dict, number of identified chunks per type
    correct_counts, true_counts, pred_counts: similar to above, but for tags
    """
    correct_chunks = defaultdict(int)
    true_chunks = defaultdict(int)
    pred_chunks = defaultdict(int)

    correct_counts = defaultdict(int)
    true_counts = defaultdict(int)
    pred_counts = defaultdict(int)

    prev_true_tag, prev_pred_tag = 0, 0
    correct_chunk = None

    for true_tag, pred_tag in zip(true_seqs, pred_seqs):
        if true_tag == pred_tag:
            correct_counts[true_tag] += 1
        true_counts[true_tag] += 1
        pred_counts[pred_tag] += 1

        _, true_type = split_tag(true_tag)
        _, pred_type = split_tag(pred_tag)

        if correct_chunk is not None:
            true_end = is_chunk_end(prev_true_tag, true_tag)
            pred_end = is_chunk_end(prev_pred_tag, pred_tag)

            if pred_end and true_end:
                correct_chunks[correct_chunk] += 1
                correct_chunk = None
            elif pred_end != true_end or true_type != pred_type:
                correct_chunk = None

        true_start = is_chunk_start(prev_true_tag, true_tag)
        pred_start = is_chunk_start(prev_pred_tag, pred_tag)

        if true_start and pred_start and true_type == pred_type:
            correct_chunk = true_type
        if true_start:
            true_chunks[true_type] += 1
        if pred_start:
            pred_chunks[pred_type] += 1

        prev_true_tag, prev_pred_tag = true_tag, pred_tag
    if correct_chunk is not None:
        correct_chunks[correct_chunk] += 1

    return (correct_chunks, true_chunks, pred_chunks, 
        correct_counts, true_counts, pred_counts)

def get_result(correct_chunks, true_chunks, pred_chunks,
    correct_counts, true_counts, pred_counts, verbose=True):
    """
    if verbose, print overall performance, as well as preformance per chunk type;
    otherwise, simply return overall prec, rec, f1 scores
    """
    # sum counts
    sum_correct_chunks = sum(correct_chunks.values())
    sum_true_chunks = sum(true_chunks.values())
    sum_pred_chunks = sum(pred_chunks.values())

    sum_correct_counts = sum(correct_counts.values())
    sum_true_counts = sum(true_counts.values())
    O_correct_counts = sum(v for k, v in correct_counts.items() if k == 0)
    O_true_counts = sum(v for k, v in true_counts.items() if k == 0)
    O_pred_counts = sum(v for k, v in pred_counts.items() if k == 0)

    chunk_types = sorted(list(set(list(true_chunks) + list(pred_chunks))))

    # compute overall precision, recall and FB1 (default values are 0.0)
    prec, rec, f1 = calc_metrics(sum_correct_chunks, sum_pred_chunks, sum_true_chunks)
    res = (prec, rec, f1)
    if not verbose:
        return res

    print("processed %i tokens with %i phrases; " % (sum_true_counts, sum_true_chunks), end='')
    print("found: %i phrases; correct: %i.\n" % (sum_pred_chunks, sum_correct_chunks), end='')
    print()
    print("%i Entity Types:" % (len(chunk_types)))
    print("accuracy: %6.2f%%; " % (100*sum_correct_counts/sum_true_counts), end='')
    print("precision: %6.2f%%; recall: %6.2f%%; FB1: %6.2f" % (prec, rec, f1))

    for t in chunk_types:
        prec, rec, f1 = calc_metrics(correct_chunks[t], pred_chunks[t], true_chunks[t])
        print("%17s: " %t , end='')
        print("precision: %6.2f%%; recall: %6.2f%%; FB1: %6.2f" %
                    (prec, rec, f1), end='')
        print("  %d" % pred_chunks[t])

    print()
    print("No Types: ")
    print("accuracy: %6.2f%%; " % (100*O_correct_counts/O_true_counts), end='')
    prec, rec, f1 = calc_metrics(O_correct_counts, O_pred_counts, O_true_counts)
    print("precision: %6.2f%%; recall: %6.2f%%; FB1: %6.2f" % (prec, rec, f1), end='')
    print("  %d" % O_pred_counts)
    return res

def ConLLEval(true_seqs, pred_seqs, verbose=True):
    (correct_chunks, true_chunks, pred_chunks,
        correct_counts, true_counts, pred_counts) = count_chunks(true_seqs, pred_seqs)
    result = get_result(correct_chunks, true_chunks, pred_chunks,
        correct_counts, true_counts, pred_counts, verbose=verbose)
    return result

## Run

In [15]:
print('[Validation Data]')
preds, truth = evaluate(valid_embeddings, valid_tags)
print()
ConLLEval(truth, preds)

[Validation Data]



Overall Accuracy: 	94.039% 	loss: 0.327

processed 16256 tokens with 661 phrases; found: 720 phrases; correct: 246.

10 Entity Types:
accuracy:  94.04%; precision:  34.17%; recall:  37.22%; FB1:  35.63
          company: precision:  12.77%; recall:  46.15%; FB1:  20.00  141
         facility: precision:  14.63%; recall:  15.79%; FB1:  15.19  41
          geo-loc: precision:  48.95%; recall:  60.34%; FB1:  54.05  143
            movie: precision:  16.67%; recall:   6.67%; FB1:   9.52  6
      musicartist: precision:  26.09%; recall:  14.63%; FB1:  18.75  23
            other: precision:   7.41%; recall:   4.55%; FB1:   5.63  81
           person: precision:  58.57%; recall:  71.93%; FB1:  64.57  210
          product: precision:  10.00%; recall:  13.51%; FB1:  11.49  50
       sportsteam: precision:  62.50%; recall:  14.29%; FB1:  23.26  16
           tvshow: precision:  11.11%; recall:  50.00%; FB1:  18.18  9

No Types: 
accuracy:  98.26%; precision:  97.40%; recall:  98.26%; FB1:  97

(34.166666666666664, 37.21633888048411, 35.62635771180304)

In [16]:
print('[Test Data]')
preds, truth = evaluate(test_embeddings, test_tags)
print()
ConLLEval(truth, preds)

[Test Data]



Overall Accuracy: 	92.154% 	loss: 0.493

processed 61880 tokens with 3473 phrases; found: 3525 phrases; correct: 1257.

10 Entity Types:
accuracy:  92.15%; precision:  35.66%; recall:  36.19%; FB1:  35.92
          company: precision:  29.69%; recall:  42.83%; FB1:  35.07  896
         facility: precision:  12.80%; recall:   8.30%; FB1:  10.07  164
          geo-loc: precision:  54.74%; recall:  60.88%; FB1:  57.65  981
            movie: precision:   6.25%; recall:   2.94%; FB1:   4.00  16
      musicartist: precision:  17.46%; recall:   5.76%; FB1:   8.66  63
            other: precision:  10.11%; recall:   6.34%; FB1:   7.79  366
           person: precision:  42.63%; recall:  71.99%; FB1:  53.55  814
          product: precision:  10.69%; recall:   5.69%; FB1:   7.43  131
       sportsteam: precision:  31.15%; recall:  12.93%; FB1:  18.27  61
           tvshow: precision:  12.12%; recall:  12.12%; FB1:  12.12  33

No Types: 
accuracy:  98.16%; precision:  96.16%; recall:  98.16%; 

(35.65957446808511, 36.19349265764469, 35.92454987139183)